# Probability of dying before 5 years

## Setup

In [1]:
from __future__ import unicode_literals, print_function, division

In [2]:
import os

In [3]:
import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

/Users/gauden/anaconda/lib/python2.7/site-packages/pytz/__init__.py:29: UserWarning: Module argparse was already imported from /Users/gauden/anaconda/lib/python2.7/argparse.pyc, but /Users/gauden/anaconda/lib/python2.7/site-packages is being added to sys.path
  from pkg_resources import resource_stream


In [4]:
import plot_constants as pc

## Specific Configuration

In [5]:
DATA_DIR = os.path.join('.', 'data')
RAW_FILE = os.path.join(DATA_DIR, 'raw_data_20150701.xlsx')

In [6]:
SHEET = 'Figure13'

PARSE_COLS = 'A:D'
SKIPROWS = 2
SKIP_FOOTER = 22-6

title = 'Probability of dying before 5 years'
subtitle = '(per 1000 live births, girls and boys, WHO European Region, 2000-2012)'
source = dict(label='European Health for All Database (HFA, DB), WHO/Europe.', 
              link=('http://www.euro.who.int/en/data-and-evidence/databases/',
                    'european-health-for-all-database-hfa-db'))

X_LABEL = 'Year'
Y_LABEL = 'Probability of dying before 5 years per 1000 live births'

## Data Importation and Munging

In [7]:
DF = pd.read_excel(RAW_FILE,
                   sheetname=SHEET,
                   parse_cols=PARSE_COLS,
                   skiprows=SKIPROWS,
                   skip_footer=SKIP_FOOTER)

DF.columns = ['year', 2000, 2005, 2012]
DF.set_index('year', inplace=True)
DF = DF.T
DF.columns = [u'Male', u'Female']
DF

,Male,Female
2000,13.97,11.14
2005,11.42,9.13
2012,9.67,7.87


## Plotting

In [8]:
plot_height = pc.PLOT_HEIGHT
plot_width = pc.PLOT_WIDTH
filename = 'vaw/fig_07'

def make_scatter(x, y, name, fill_color, line_color, symbol):
    trace = Scatter(x=x,
                    y=y,
                    mode='markers+lines',
                    name=name,
                    marker=Marker(color=fill_color,
                                  line=Line(color=line_color,
                                            width=1,
                                           ),
                                  symbol=symbol,
                                  size=12,)
                   )
    return trace

female_trace = make_scatter(x=DF.index,
                            y=DF.Female,
                            name='Female',
                            fill_color=pc.FEMALE_FILL,
                            line_color=pc.FEMALE_LINE,
                            symbol=pc.FEMALE_SYMBOL)

male_trace = make_scatter(x=DF.index,
                          y=DF.Male,
                          name='Male',
                          fill_color=pc.MALE_FILL,
                          line_color=pc.MALE_LINE,
                          symbol=pc.MALE_SYMBOL)

data = Data([female_trace, male_trace])

layout = Layout(
    title = '<b>{}</b><br>{}'.format(title, subtitle),
    autosize=False,
    width=plot_width,
    height=plot_height,
    margin=Margin(
        l=150,
        r=120,
        pad=0
    ),
    xaxis=XAxis(title=X_LABEL),
    yaxis=YAxis(title=Y_LABEL,
               range=[0,DF.Male.max()+1]),
    annotations=Annotations([
        Annotation(
            x=0.5,
            y=-0.1,
            xref='paper',
            yref='paper',
            text='Source: <a href="{}">{}</a>'.format(source['link'],source['label']),
            align='center',
            showarrow=False
        )
    ]),
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename=filename, height=plot_height)